In [ ]:
include("ecow.jl")

using Plots
const mm = Plots.mm
fntsm = Plots.font("times", 12)
fntlg = Plots.font("times", 14)
default(
    titlefont=fntlg,
    guidefont=fntlg,
    tickfont=fntsm,
    legendfont=fntsm,
    left_margin=3mm,
    bottom_margin=10mm,
    top_margin=0mm,
    right_margin=0mm,
    size=(500,300),
    xlabel="Days of the Season\n",
    label="",
    w=3,
    c="#00467F"
)
gr()

In [ ]:
cow = ECow.Cow()
herbage = ECow.FoodOffer(ECow.Pasture(), 35.0)
supplement = ECow.FoodOffer(ECow.PKE(), 2.0)

milkingplan(t,n=365) = vcat(trues(t), falses(n-t))

results = Dict(
    :bcs         => vcat(3.2, zeros(365)),
    :maintenance => vcat(63, zeros(365)),
    :lactation   => milkingplan(40 * 7),  # milk for 40 weeks
    :supplement  => zeros(365),
    :herbage     => zeros(365),
    :milksolids  => zeros(365)
)

for day in 1:365
    (new_bcs, new_maintenance, actual_herbage_intake, actual_supplement_intake, milksolids) = ECow.updateday!(
        cow, day, results[:bcs][day], results[:maintenance][day], results[:lactation][day], supplement, herbage
    )
    results[:bcs][day+1]         = new_bcs
    results[:maintenance][day+1] = new_maintenance
    results[:supplement][day]    = actual_supplement_intake
    results[:herbage][day]       = actual_herbage_intake
    results[:milksolids][day]    = milksolids
end

bcs_plot = plot(results[:bcs],
    ylabel="BCS (Units)",
    title="(a)",
    ylims=(0,5)
)

mai_plot = plot(results[:maintenance],
    ylabel="Maintenance (MJ/Day)",
    title="(b)",
    ylims=(50,80)
)

her_plot = plot(
    ylabel="Feed Intake (kg/Day)",
    title="(c)",
    ylims=(0,14),
    legend=:topleft
)
plot!(results[:herbage], label="Herbage")
plot!(results[:supplement], label="Supplement", linestyle=:dash)

mlk_plot = plot(results[:milksolids],
    ylabel="Milk Solids (kg/Day)",
    title="(d)",
    ylims=(0,1.5)
)

plot(bcs_plot, mai_plot, her_plot, mlk_plot, size=(1000, 600))

savefig("moo_example.pdf")

In [ ]:
RESULTS = "moo.results"
using JSON
results = JSON.parsefile(RESULTS)
summary = ECow.summarize(3.2, 70.0, results["supplement"][1], results["herbage"][1], results["lactation"][1])
plot(
    plot(summary[:bcs],
        title="(a)", ylabel="BCS (Units)",
        ylims=(0, 5)
    ),
    plot(summary[:maintenance],
        title="(b)", ylabel="Maintenance (MJ/day)",
        ylims=(50, 80)
    ),
    plot(summary[:lactation],
        title="(c)", ylabel="Lactation",
        ylims=(0, 1)
    ),
    plot(summary[:herbage],
        title="(d)", ylabel="Herbage (kg/cow/day)",
        ylims=(0, 15)
    ),
    plot(summary[:supplement],
        title="(e)", ylabel="Supplement (kg/cow/day)",
        ylims=(0, 5)
    ),
    plot(summary[:milksolids],
        title="(g)", ylabel="Milk Solids (kg/cow/day)",
        ylims=(0, 2)
    ),
    size=(1500,600)
)
savefig("simulation.pdf")

In [ ]:
SOLVED_MODEL = "moo.solved.model"

using DynamicProgramming
m = open(SOLVED_MODEL, "r") do io
   deserialize(io) 
end

function plotvaluetogo(stage, title="")
    X = 2.5:0.1:4.5
    Y = 2000:50:3000
    stage_one_surface = [
        m.stages[stage].interpolatedsurface[bcs,70.0,1.0,cover]
        for bcs in X, cover in Y
    ]
    contour(X, Y, stage_one_surface',
        title        = title,
        xlabel       = "BCS\n",
        ylabel       = "Pasture Cover (kg/Ha)",
        right_margin = 7.5mm,
        left_margin  = 5mm,
        levels       = 20
    )
end
plot(
    plotvaluetogo(1, "(a) Week 1"),
    plotvaluetogo(25, "(b) Week 25"),
    plotvaluetogo(50, "(c) Week 50"),
    size=(1500, 300), layout=(1,3)
)
savefig("value-to-go.pdf")